<h1> <center> <font color="blue"> AdaBoost </center></h1>

<center><img src="pics/Training-of-an-AdaBoost-classifier-The-first-classifier-trains-on-unweighted-data-then.png" width=600></center>

<link> Source : https://www.researchgate.net/figure/Training-of-an-AdaBoost-classifier-The-first-classifier-trains-on-unweighted-data-then_fig3_306054843 <\link>

<h3> <font color="blue"> Pros </h3>

<h3> <font color = "red"> Cons </h3>

<h3> <font color = "green"> How it works </h3>

1. Initially all training samples are given equal weights. So if we have 100 training samples, the sample weight will be 1/00 for EACH of the training sample.

2. Create a Decision Stump. This will be Decision Tree with one node only.
Calcuate the number of correct and incorrect predictions (MSE in case of Regression)
Suppose we had 20 incorrect and 80 correct.</p2>

3. Calculate by following formula</p2>

    Significance = 1 / 2 (log (1-TotalError)/TotalError)
    
    TotalError = 20 / 100
    
    Significance = 0.69
    
4. For Incorrect Observation : New Sample Weights = Sample Weight * e ^ Significance = 0.2 * 1.99 = 0.398
    
5. For Correct Observations : New Sample Weights = Sample Weight * e ^ - Significance = 0.2 * 0.50 = 0.10
    
6. Idea is to **Penalize**. wrong classification , while **depanelize** correct classification.

7. This Step will Continue until the weights can not be gone down.
    

In [10]:
import numpy as np

np.exp(-0.69)

0.5015760690660556

In [11]:
0.2 * 0.50

0.1

<h3> <span style="background:yellow"> Hyper-parameters </h3>